In [1]:
!pip install tensorflow-gpu==2.0.0

     |████████████████████████████████| 380.8MB 47kB/s 
     |████████████████████████████████| 450kB 25.9MB/s 
     |████████████████████████████████| 3.8MB 42.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=c10db212381390f67adffb9d324f920b920ce9855439a694911a3209cfe36a8f
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc3 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2

In [0]:
!pip install keras==2.3.1

In [3]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
import numpy as np
import pandas as pd
import re

In [0]:
data_file1 = open("alchemist.txt", "r", encoding='utf8')
data_file2 = open("metamorphosis.txt", "r", encoding='utf8')

In [0]:
data1 = data_file1.read()
data2 = data_file2.read()

In [16]:
print("Length of Book 1 : ", len(data1))
print("Length of Book 2 : ", len(data2))

Length of Book 1 :  212244
Length of Book 2 :  119164


In [0]:
def filterAlpha(text):
    text = text.replace('\n', ' ')
    s_result = []
    for i in text:
        if i.isalpha() or i == ' ':
            s_result.append(i.lower())
    return ''.join(s_result)

In [18]:
def removeSpecialChars(data):
  data = filterAlpha(data)
  data = re.sub(' +', ' ', data)
  data = data.strip()
  return data

data1 = removeSpecialChars(data1)
data2 = removeSpecialChars(data2)

#After removing punctuations
print("Length of Book 1 : ", len(data1))
print("Length of Book 2 : ", len(data2))

Length of Book 1 :  200354
Length of Book 2 :  115348


In [0]:
tokens1 = data1.split()
tokens2 = data2.split()

In [0]:
vocab = set(tokens1 + tokens2)

In [23]:
len(vocab)

4637

In [0]:
length = 50 + 1
lines = []
for i in range(length, len(tokens1)):
    seq = tokens[i-length : i]
    line = ' '.join(seq)
    lines.append(line)
for i in range(length, len(tokens2)):
    seq = tokens[i-length : i]
    line = ' '.join(seq)
    lines.append(line)

In [25]:
len(lines)

60750

In [27]:
lines[0]

'the boys name wassantiago dusk was falling as the boy arrived with his herd at an abandoned church the roof had fallen in long ago and an enormous sycamore had grown on the spot where the sacristy had once stood he decided to spend the night there he saw to it'

In [0]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
tokenizer = Tokenizer()

In [0]:
tokenizer.fit_on_texts(lines)

In [0]:
sequences = tokenizer.texts_to_sequences(lines)

In [0]:
sequences = np.array(sequences)

In [33]:
print(len(sequences))
print(sequences)

60750
[[   1  214  369 ...  151    2   13]
 [ 214  369 3316 ...    2   13    7]
 [ 369 3316 1696 ...   13    7   39]
 ...
 [ 535  182  648 ...    1 2675    5]
 [ 182  648    3 ... 2675    5 1056]
 [ 648    3   16 ...    5 1056    4]]


In [0]:
#sequences[0, :-1]
X, y = sequences[:, :-1], sequences[:, -1]

In [35]:
X[0]

array([   1,  214,  369, 3316, 1696,    8, 2673,   21,    1,   12,  653,
         20,   11, 3314,   22,   62, 1492,  706,    1, 1188,    9,  652,
         10,  255,  431,    4,   62,  841,  840,    9, 1496,   32,    1,
       1694,   87,    1, 1494,    9,  191,  520,    3,  190,    2,  476,
          1,  155,   29,    3,  151,    2])

In [36]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

3318


In [0]:
y = to_categorical(y, num_classes = vocab_size)

In [0]:
seq_length = X.shape[1]

In [39]:
seq_length

50

In [0]:
model = Sequential()

In [0]:
model.add(Embedding(vocab_size, 50, input_length=seq_length))

In [0]:
model.add(LSTM(125, return_sequences=True, dropout=0.2))
model.add(LSTM(125, dropout=0.2))

In [0]:
model.add(Dense(100, activation='relu'))

In [0]:
model.add(Dense(vocab_size, activation='softmax'))

In [78]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 50)            165900    
_________________________________________________________________
lstm_6 (LSTM)                (None, 50, 125)           88000     
_________________________________________________________________
lstm_7 (LSTM)                (None, 125)               125500    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               12600     
_________________________________________________________________
dense_3 (Dense)              (None, 3318)              335118    
Total params: 727,118
Trainable params: 727,118
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss ='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [80]:
history = model.fit(X, y, batch_size=128, epochs=100)

Train on 60750 samples
Epoch 1/100
60750/60750 [==============================] - 25s 415us/sample - loss: 6.0739 - accuracy: 0.0902
Epoch 2/100
60750/60750 [==============================] - 21s 350us/sample - loss: 5.5717 - accuracy: 0.1088
Epoch 3/100
60750/60750 [==============================] - 21s 348us/sample - loss: 5.3451 - accuracy: 0.1279
Epoch 4/100
60750/60750 [==============================] - 21s 351us/sample - loss: 5.1847 - accuracy: 0.1363
Epoch 5/100
60750/60750 [==============================] - 21s 347us/sample - loss: 5.0680 - accuracy: 0.1437
Epoch 6/100
60750/60750 [==============================] - 21s 347us/sample - loss: 4.9420 - accuracy: 0.1491
Epoch 7/100
60750/60750 [==============================] - 21s 348us/sample - loss: 4.8233 - accuracy: 0.1570
Epoch 8/100
60750/60750 [==============================] - 21s 347us/sample - loss: 4.7442 - accuracy: 0.1615
Epoch 9/100
60750/60750 [==============================] - 21s 347us/sample - loss: 4.6849 - accu

In [0]:
seed_text = lines[20]
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
    text = []
    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen=text_seq_length, truncating='pre')
        y_pred = model.predict_classes(encoded)
        predicted_word = ''
        for word, index in tokenizer.word_index.items():
            if index == y_pred:
                predicted_word = word
                break
        seed_text = seed_text + ' ' + predicted_word
        text.append(predicted_word)
    return ' '.join(text)

In [82]:
#seed_text = "Once there was an old man who had four sons <eos> All four of them were very lazy <eos> One day the old man fell sick and was counting his last days in bed <eos> He worried a lot about his sons future as the young men hesitated a lot"
#seed_text = seed_text.lower()
print(seed_text)

had fallen in long ago and an enormous sycamore had grown on the spot where the sacristy had once stood he decided to spend the night there he saw to it that all the sheep entered through the ruined gate and then laid some planks across it to prevent the flock


In [83]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)

'from wandering to the window the camel driver understood nature had been a party but had caused him to do you am in the middle of the desert and the woman told him about several hours meanwhile i have to worry she went to the old king and all the sheep he had discovered before the presence of the caravan and converse with him and purpose and felt as they reached to him but the boy had to be the same stars and the boy spoke to the boy and they were in from rock places he thought to the'

In [68]:
lines[70]

'flock from wandering away during the night there were no wolves in the region but once an animal had strayed during the night and the boy had had to spend the entire next day searching for it he swept the floor with his jacket and lay down using the book he'

In [0]:
seed_text = "he boy had dreamed of the treasure he had no idea where the treasure is but he was sure it was somewhere hidden in the mountains"

In [85]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)

'just to see the language of the world and the boy couldnt understand what he was going to have to pay anything to mecca the boy said but i could have to learn it to you that your flock across the fields it always like to change the future i should be a pleasant surprise he told the bar one night the boy looked at the east to chant a small place and all the boy and presented him food and onto the boy was shocked the old man knew how to read the boys thoughts just as he himself'

In [0]:
model.save('modeltrained3.h5')

In [0]:
import pickle
# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)